In [1]:
from lightgbm import LGBMRegressor
import lightgbm as lgb
import pandas as pd
import re
from sklearn.metrics import mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import holidays
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from torch.optim.lr_scheduler import StepLR
from torch.nn.utils import clip_grad_norm_

In [2]:
#!pip install holidays

In [3]:
def find_season(month, hemisphere):
    if hemisphere == 'Southern':
        season_month_south = {
            12:'Summer', 1:'Summer', 2:'Summer',
            3:'Autumn', 4:'Autumn', 5:'Autumn',
            6:'Winter', 7:'Winter', 8:'Winter',
            9:'Spring', 10:'Spring', 11:'Spring'}
        return season_month_south.get(month)
        
    elif hemisphere == 'Northern':
        season_month_north = {
            12:'Winter', 1:'Winter', 2:'Winter',
            3:'Spring', 4:'Spring', 5:'Spring',
            6:'Summer', 7:'Summer', 8:'Summer',
            9:'Autumn', 10:'Autumn', 11:'Autumn'}
        return season_month_north.get(month)
    else:
        print('Invalid selection. Please select a hemisphere and try again')

In [4]:
data = pd.read_csv('../train_group2/merged_data.csv', skiprows = 0)

#data['Unnamed: 0'] = pd.to_datetime(data['Unnamed: 0'])
#data.set_index(data.columns[0], inplace=True)
#data.index.name = 'Time'

data = data.rename(columns={'Unnamed: 0': 'datetime'})

data['datetime'] = pd.to_datetime(data['datetime'])


# Drop the first column
data = data.drop(data.columns[1], axis=1)

In [5]:
data['month'] = data['datetime'].dt.month # Which month of the corresponding year
data['day_of_month'] = data['datetime'].dt.day # Which day of the corresponding month
#data['day_of_year'] = data['datetime'].dt.dayofyear # Which day of the corresponding year
#data['week_of_year'] = data['datetime'].dt.weekofyear # Which week of the corresponding year
data['day_of_week'] = data['datetime'].dt.dayofweek # Which day of the corresponding week of the each month
data["quarter"] = data['datetime'].dt.quarter
data["is_wknd"] = data['datetime'].dt.weekday // 4 # df.date.dt.weekday => Starts from '0' means '0' = 'Monday'. So, '// 4' will give '1' when day number equals
# to '5'(which corresponds 'Saturday') and '6'(which corresponds 'Sunday') and '0' for rest of them. Consequently this column will represent whether 
# the day is weekend or not
data['is_month_start'] = data['datetime'].dt.is_month_start.astype(int) # Is it starting of the corresponding month
data['is_month_end'] = data['datetime'].dt.is_month_end.astype(int)
data['hour'] = data['datetime'].dt.hour
data['var'] = data['MWh'].rolling(window=24*5).var()

season_list = []
hemisphere = 'Northern'
for month in data['month']:
    season = find_season(month, hemisphere)
    season_list.append(season)
    
data['season'] = season_list



In [6]:
countries = {
    'CH': 'Switzerland',
    'IT': 'Italy',
    'FR': 'France',
    'GER': 'Germany',
}

# For each country, add a new column with holiday names or NaN if not a holiday
for code, country in countries.items():
    holiday_dict = holidays.CountryHoliday(country, years=data['datetime'].dt.year.unique().tolist())
    data[country] = data['datetime'].apply(lambda x: 1 if x in holiday_dict else 0)



In [7]:
# Perform one-hot encoding on 'month' and 'weekday'

df_encoded = pd.get_dummies(data, columns=['Switzerland','Italy', 'France','Germany' ,'month', 'day_of_month', "day_of_week", "quarter", "is_wknd", 'is_month_start','is_month_end', 'season' ], prefix=['Switzerland','Italy', 'France','Germany' ,'month', 'day_of_month', "day_of_week", "quarter", "is_wknd", 'is_month_start','is_month_end', 'season'])

# remove datetime
df_encoded = df_encoded.drop('datetime', axis=1)


#for col in ['month', 'day_of_month', "day_of_week", "quarter", "is_wknd", 'is_month_start','is_month_end', 'season' ]:
#    df_encoded[col] = df_encoded[col].astype(bool)


In [8]:
import numpy as np

for col in df_encoded.columns:
    if df_encoded[col].dtype == 'object':
        # Assuming the column's values can be converted to boolean
        df_encoded[col] = df_encoded[col].astype(bool)

data = df_encoded

# Define sequence lengths
input_seq_length = 7 * 24  # Previous 7 days
output_seq_length = 24     # Next 24 hours

# Initialize lists to store sequences
input_sequences = []
output_sequences = []

# Iterate over the data to create sequences
for i in range(0, len(data) - input_seq_length - output_seq_length + 1, output_seq_length):
    input_seq = data.iloc[i : i + input_seq_length + output_seq_length]
    input_seq.iloc[input_seq_length + output_seq_length -1-24 :input_seq_length + output_seq_length - 1]['MWh'] = np.nan
    output_seq = data.iloc[i + input_seq_length : i + input_seq_length + output_seq_length]['MWh']
    
    # Append the sequences to the lists
    input_sequences.append(input_seq)
    output_sequences.append(output_seq)

# Convert lists to NumPy arrays
input_sequences = np.array(input_sequences)
output_sequences = np.array(output_sequences)
        
        


/tmp/ipykernel_390886/3602654270.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_seq.iloc[input_seq_length + output_seq_length -1-24 :input_seq_length + output_seq_length - 1]['MWh'] = np.nan


In [9]:
input_sequences.shape

(1089, 192, 91)

In [10]:
output_sequences.shape

(1089, 24)

In [11]:
min_max_scaler_data = MinMaxScaler()
input_data_reshaped = input_sequences.reshape(-1, 1)  # Scaler expects 2D data
normalized_input_data = min_max_scaler_data.fit_transform(input_data_reshaped)
normalized_input_data = normalized_input_data.reshape(input_sequences.shape)

min_max_scaler_labels = MinMaxScaler()
normalized_labels = min_max_scaler_labels.fit_transform(output_sequences)

input_sequences = normalized_input_data
output_sequences = normalized_labels

In [12]:
# Calculate the index to split the data
split_index = int(len(input_sequences) * 0.97)


# Split the data into training and testing sets
input_train = input_sequences[:split_index]

output_train = output_sequences[:split_index]
input_test = input_sequences[split_index:]
output_test = output_sequences[split_index:]

# Ensure the data types are compatible with PyTorch tensors
input_train = input_train.reshape(input_train.shape[0], input_train.shape[1] * input_train.shape[2])
#output_train = output_train.astype(np.float32)
input_test = input_test.reshape(input_test.shape[0], input_test.shape[1] * input_test.shape[2])

#output_test = output_test.astype(np.float32)



"""
# Convert NumPy arrays to PyTorch tensors
input_train_tensor = torch.tensor(input_train, dtype=torch.float32)
output_train_tensor = torch.tensor(output_train, dtype=torch.float32)
input_test_tensor = torch.tensor(input_test, dtype=torch.float32)
output_test_tensor = torch.tensor(output_test, dtype=torch.float32)

# Create PyTorch datasets
train_dataset = TensorDataset(input_train_tensor, output_train_tensor)
test_dataset = TensorDataset(input_test_tensor, output_test_tensor)

# Define batch size
batch_size = 64  # Adjust as needed

# Create PyTorch dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

"""



'\n# Convert NumPy arrays to PyTorch tensors\ninput_train_tensor = torch.tensor(input_train, dtype=torch.float32)\noutput_train_tensor = torch.tensor(output_train, dtype=torch.float32)\ninput_test_tensor = torch.tensor(input_test, dtype=torch.float32)\noutput_test_tensor = torch.tensor(output_test, dtype=torch.float32)\n\n# Create PyTorch datasets\ntrain_dataset = TensorDataset(input_train_tensor, output_train_tensor)\ntest_dataset = TensorDataset(input_test_tensor, output_test_tensor)\n\n# Define batch size\nbatch_size = 64  # Adjust as needed\n\n# Create PyTorch dataloaders\ntrain_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)\ntest_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)\n\n'

In [13]:
input_sequences.shape

(1089, 192, 91)

In [14]:
"""
# Ensure the data types are compatible with PyTorch tensors
all_input = input_sequences.astype(np.float32)
all_output = output_sequences.astype(np.float32)

# Convert NumPy arrays to PyTorch tensors
all_input_train_tensor = torch.tensor(all_input, dtype=torch.float32)
all_output_train_tensor = torch.tensor(all_output, dtype=torch.float32)

# Create PyTorch datasets
full_dataset = TensorDataset(all_input_train_tensor, all_output_train_tensor)

# Define batch size
batch_size = 64  # Adjust as needed

# Create PyTorch dataloaders
full_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)
"""

'\n# Ensure the data types are compatible with PyTorch tensors\nall_input = input_sequences.astype(np.float32)\nall_output = output_sequences.astype(np.float32)\n\n# Convert NumPy arrays to PyTorch tensors\nall_input_train_tensor = torch.tensor(all_input, dtype=torch.float32)\nall_output_train_tensor = torch.tensor(all_output, dtype=torch.float32)\n\n# Create PyTorch datasets\nfull_dataset = TensorDataset(all_input_train_tensor, all_output_train_tensor)\n\n# Define batch size\nbatch_size = 64  # Adjust as needed\n\n# Create PyTorch dataloaders\nfull_loader = DataLoader(full_dataset, batch_size=batch_size, shuffle=True)\n'

In [15]:
#output_train[:,0]

# LightGBM parameters : Actually that parameters is pretrained parameters means this parameters have been found by GridSearch/RandomizedSearch. 
#So I am writing using it directly, normally it should have been found by GridSearch/RandomizedSearch methods. 
lgb_params = {'metric': {'mae'},
              'num_leaves': 10,
              'learning_rate': 0.02,
              'feature_fraction': 0.8,
              'max_depth': 5,
              'verbose': 0,
              'early_stopping_rounds': 200,
             'device': 'gpu'}

pattern = re.compile(r'[,\[\]{}"]')

# Check each column name for special characters
problematic_columns = [col for col in X_train.columns if pattern.search(col)]

print(problematic_columns)


In [16]:
"""
lgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)
lgbval = lgb.Dataset(data=X_val, label=Y_val, reference=lgbtrain, feature_name=cols)

model = lgb.train(lgb_params, lgbtrain,
                  valid_sets=[lgbtrain, lgbval], 
                 num_boost_round = 11000)
                 
"""

'\nlgbtrain = lgb.Dataset(data=X_train, label=Y_train, feature_name=cols)\nlgbval = lgb.Dataset(data=X_val, label=Y_val, reference=lgbtrain, feature_name=cols)\n\nmodel = lgb.train(lgb_params, lgbtrain,\n                  valid_sets=[lgbtrain, lgbval], \n                 num_boost_round = 11000)\n                 \n'

In [17]:

"""
model = lgb.LGBMRegressor(**lgb_params)
model.fit(X_train, Y_train, eval_set=[(X_val, Y_val)])
predictions = model.predict(X_val)
rmse = mean_absolute_error(Y_val, predictions)
print(rmse)
"""

'\nmodel = lgb.LGBMRegressor(**lgb_params)\nmodel.fit(X_train, Y_train, eval_set=[(X_val, Y_val)])\npredictions = model.predict(X_val)\nrmse = mean_absolute_error(Y_val, predictions)\nprint(rmse)\n'

In [18]:
"""
# LGBM
cv_split = TimeSeriesSplit(n_splits=4, test_size=100)
model = lgb.LGBMRegressor()
parameters = {
    "max_depth": [3, 4, 6, 5, 10],
    "num_leaves": [10, 20, 30, 40, 100, 120],
    "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],
    "n_estimators": [50, 100, 300, 500, 700, 900, 1000],
    "colsample_bytree": [0.3, 0.5, 0.7, 1]
}


grid_search = GridSearchCV(estimator=model, cv=cv_split, param_grid=parameters)
grid_search.fit(X_train, y_train)
"""

'\n# LGBM\ncv_split = TimeSeriesSplit(n_splits=4, test_size=100)\nmodel = lgb.LGBMRegressor()\nparameters = {\n    "max_depth": [3, 4, 6, 5, 10],\n    "num_leaves": [10, 20, 30, 40, 100, 120],\n    "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],\n    "n_estimators": [50, 100, 300, 500, 700, 900, 1000],\n    "colsample_bytree": [0.3, 0.5, 0.7, 1]\n}\n\n\ngrid_search = GridSearchCV(estimator=model, cv=cv_split, param_grid=parameters)\ngrid_search.fit(X_train, y_train)\n'

In [19]:

lgb_params = {
    'metric': 'mae',
    'num_leaves': 10,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'max_depth': 20,
    'verbose': 1,  # changed to 1 for verbose logging
    'num_boost_round': 300,
    'early_stopping_rounds': 100,
    'device': 'gpu',
    # 'gpu_platform_id': 0,  # Uncomment if necessary
    # 'gpu_device_id': 0,    # Uncomment if necessary
    'gpu_use_dp': True  # Use double precision
}

models = {}
model_multistep = lgb.LGBMRegressor(**lgb_params)  # Initialize once

for i in range(24):
    print("step:", i)
    
    # Assuming input_train, output_train, input_test, and output_test are already defined
    model_multistep.fit(input_train, output_train[:, i], eval_set=[(input_test, output_test[:, i])])  # Print every 100 rounds
    
    models[f'model_{i}'] = model_multistep
    


step: 0


/opt/tljh/user/lib/python3.9/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] num_iterations is set=300, num_boost_round=300 will be ignored. Current value: num_iterations=300
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 578536
[LightGBM] [Info] Number of data points in the train set: 1056, number of used features: 16792
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 3704 dense feature groups (3.73 MB) transferred to GPU in 0.0

/opt/tljh/user/lib/python3.9/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 578536
[LightGBM] [Info] Number of data points in the train set: 1056, number of used features: 16792
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 3704 dense feature groups (3.73 MB) transferred to GPU in 0.021696 secs. 1 sparse feature groups
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] num_iterations is set=300, num_boost_round=300 will be ignored. Current value: num_iterations=300
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Curr

...

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 578536
[LightGBM] [Info] Number of data points in the train set: 1056, number of used features: 16792
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 3704 dense feature groups (3.73 MB) transferred to GPU in 0.023196 secs. 1 sparse feature groups
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] num_iterations is set=300, num_boost_round=300 will be ignored. Current value: num_iterations=300
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Curr

/opt/tljh/user/lib/python3.9/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 578536
[LightGBM] [Info] Number of data points in the train set: 1056, number of used features: 16792
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 3704 dense feature groups (3.73 MB) transferred to GPU in 0.023117 secs. 1 sparse feature groups
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] num_iterations is set=300, num_boost_round=300 will be ignored. Current value: num_iterations=300
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Curr

/opt/tljh/user/lib/python3.9/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 578536
[LightGBM] [Info] Number of data points in the train set: 1056, number of used features: 16792
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 16
[LightGBM] [Info] 3704 dense feature groups (3.73 MB) transferred to GPU in 0.022850 secs. 1 sparse feature groups
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] num_iterations is set=300, num_boost_round=300 will be ignored. Current value: num_iterations=300
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Curr

In [21]:
import pickle

with open('saji_models_v2.pkl', 'wb') as f:
    pickle.dump(models, f)

In [22]:
np.mean(abs(models['model_0'].predict(input_test) - output_test[:, 0]))
np.mean(abs(models['model_1'].predict(input_test) - output_test[:, 1]))
np.mean(abs(models['model_2'].predict(input_test) - output_test[:, 2]))
np.mean(abs(models['model_3'].predict(input_test) - output_test[:, 3]))
np.mean(abs(models['model_4'].predict(input_test) - output_test[:, 4]))
np.mean(abs(models['model_5'].predict(input_test) - output_test[:, 5]))
np.mean(abs(models['model_6'].predict(input_test) - output_test[:, 6]))

np.mean(abs(models['model_7'].predict(input_test) - output_test[:, 7]))
np.mean(abs(models['model_8'].predict(input_test) - output_test[:, 8]))
np.mean(abs(models['model_9'].predict(input_test) - output_test[:, 9]))
np.mean(abs(models['model_10'].predict(input_test) - output_test[:, 10]))
np.mean(abs(models['model_11'].predict(input_test) - output_test[:, 11]))
np.mean(abs(models['model_12'].predict(input_test) - output_test[:, 12]))
np.mean(abs(models['model_13'].predict(input_test) - output_test[:, 13]))

np.mean(abs(models['model_14'].predict(input_test) - output_test[:, 14]))
np.mean(abs(models['model_15'].predict(input_test) - output_test[:, 15]))
np.mean(abs(models['model_16'].predict(input_test) - output_test[:, 16]))
np.mean(abs(models['model_17'].predict(input_test) - output_test[:, 17]))
np.mean(abs(models['model_18'].predict(input_test) - output_test[:, 18]))
np.mean(abs(models['model_19'].predict(input_test) - output_test[:, 19]))
np.mean(abs(models['model_20'].predict(input_test) - output_test[:, 20]))

np.mean(abs(models['model_21'].predict(input_test) - output_test[:, 21]))
np.mean(abs(models['model_22'].predict(input_test) - output_test[:, 22]))
np.mean(abs(models['model_23'].predict(input_test) - output_test[:,22]))


[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGB

0.03803828786876852

In [33]:
output_test.shape

(33, 24)

In [85]:
output_test

array([[116.60924408, 126.9984372 , 125.95285102, ..., 126.81711829,
        107.85424371,  89.2776395 ],
       [ 82.13964682,  82.59810723,  93.61280993, ...,  89.39895988,
         83.58139282,  93.70273729],
       [ 99.32936341,  95.81622047,  90.21143037, ...,  67.05376591,
         64.2172988 ,  66.06269023],
       ...,
       [124.12143695, 115.54929113, 108.69566134, ..., 205.33900331,
        198.81006446, 177.54828634],
       [133.9305803 , 132.4915909 , 136.87537298, ..., 193.91019451,
        189.60759661, 161.07037915],
       [176.1359428 , 193.20642619, 198.78469971, ..., 155.10951973,
        171.37027694, 146.05479082]])

In [27]:
single_prediction

NameError: name 'single_prediction' is not defined

In [30]:
predictions

NameError: name 'predictions' is not defined

In [ ]:
def mean_absolute_errors(models, input_test, output_test):
    errors = []
    aggregated_predictions = np.zeros((input_test.shape[0], 24))  # Assuming input_test.shape[0] = 33

    for i in range(24):
        predictions = models[f'model_{i}'].predict(input_test)
        aggregated_predictions[:, i] = predictions
        
        
        
        error = np.mean(abs(predictions - output_test[:, i]))
        errors.append(error)

    return np.mean(errors), aggregated_predictions



In [31]:
def mean_absolute_errors(models, input_test, output_test):
    errors = []
    for i in range(24):
        predictions = models[f'model_{i}'].predict(input_test)
        print(predictions.shape)
        error = np.mean(abs(predictions - output_test[:, i]))
        print(error)
        errors.append(error)
    return np.mean(errors)

overall_mean = mean_absolute_errors(models, input_test, output_test)
print(f"Overall mean of the MAEs: {overall_mean}")

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
(33,)
0.10414416255407417
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
(33,)
0.09819737510275117
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
(33,)
0.08306418273447422
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
(33,)
0.08314739722347197
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
(33,)
0.09052900921557827
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
(33,)
0.07919659010578807
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current valu

In [53]:
# Enter here location to your test data and modell
activelosses = "../results_group2/Active-Losses-2022_test.csv"
renewablegen = "../results_group2/Forecast-renewable-generation_2022_test.csv"
forecasttemp = '../results_group2/eq_temp_2022_test.csv.csv'
ntc = '../results_group2/ntc_2022_test.csv'

In [54]:
activeslosses = pd.read_csv(activelosses)
activeslosses.drop(["Unnamed: 0.1", "Unnamed: 0"], axis = 1)
activeslosses = activeslosses.iloc[:, 2:]
# Rename column 'A' to 'X'
activeslosses['datetime'] = pd.to_datetime(activeslosses['datetime'])
activeslosses.rename(columns={'Wirkverluste/Active Losses': 'MWh'}, inplace=True)
activeslosses.set_index(activeslosses.columns[0], inplace=True)
activeslosses = activeslosses[~activeslosses.index.duplicated(keep='first')]
activeslosses

,MWh
datetime,
2022-01-01 00:00:00,146.054792
2022-01-01 01:00:00,139.133354
2022-01-01 02:00:00,147.562500
2022-01-01 03:00:00,157.636204
2022-01-01 04:00:00,163.326766
...,...
2022-12-31 19:00:00,67.876028
2022-12-31 20:00:00,72.765318
2022-12-31 21:00:00,81.277633


In [55]:
Forecast_renew = pd.read_csv(renewablegen, skiprows=0)
Forecast_renew = Forecast_renew.drop(["Unnamed: 0.1", "Unnamed: 0"], axis = 1)
# Load Forecast Renewable Generation data
Forecast_renew['datetime'] = pd.to_datetime(Forecast_renew['datetime'])
Forecast_renew.set_index(Forecast_renew.columns[0], inplace=True)
Forecast_renew = Forecast_renew[~Forecast_renew.index.duplicated(keep='first')]
Forecast_renew = Forecast_renew.resample('H').asfreq()
Forecast_renew

,solar_fore_de_mw,solar_fore_it_mw,wind_fore_de_mw,wind_fore_it_mw
datetime,,,,
2022-01-01 00:00:00,0.0,0.0,31805.65,1331.48
2022-01-01 01:00:00,0.0,0.0,29880.67,1438.15
2022-01-01 02:00:00,0.0,0.0,28826.75,1623.80
2022-01-01 03:00:00,0.0,0.0,27631.75,1894.75
2022-01-01 04:00:00,0.0,0.0,27128.00,2335.05
...,...,...,...,...
2022-12-31 19:00:00,0.0,0.0,44107.32,569.68
2022-12-31 20:00:00,0.0,0.0,44512.60,459.45
2022-12-31 21:00:00,0.0,0.0,44946.45,399.35


In [56]:
Forecast_temp = pd.read_csv(forecasttemp, skiprows=0)
Forecast_temp = Forecast_temp.drop(["Unnamed: 0.1", "Unnamed: 0"], axis = 1)
Forecast_temp['datetime'] = pd.to_datetime(Forecast_temp['datetime'])
Forecast_temp.set_index(Forecast_temp.columns[0], inplace=True)
Forecast_temp = Forecast_temp[~Forecast_temp.index.duplicated(keep='first')]
Forecast_temp

,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it
datetime,,,,
2022-01-01 00:00:00,6.65,7.90,10.25,7.38
2022-01-01 01:00:00,6.77,8.31,10.64,7.25
2022-01-01 02:00:00,6.42,8.01,10.46,7.09
2022-01-01 03:00:00,6.08,7.77,10.21,6.94
2022-01-01 04:00:00,5.68,7.62,10.07,6.85
...,...,...,...,...
2022-12-31 19:00:00,10.37,14.18,14.13,11.68
2022-12-31 20:00:00,9.67,13.90,13.91,10.76
2022-12-31 21:00:00,9.10,13.57,13.79,10.06


In [57]:
# Load NTC data
NTC = pd.read_csv(ntc, skiprows=0)
NTC = NTC.drop(["Unnamed: 0.1", "Unnamed: 0"], axis = 1)
NTC['datetime'] = pd.to_datetime(NTC['datetime'])
NTC.set_index(NTC.columns[0], inplace=True)
NTC = NTC[~NTC.index.duplicated(keep='first')]
NTC = NTC.resample('H').asfreq()
NTC = NTC.interpolate(method='polynomial', order=2)
NTC

,CH_AT,CH_DE,CH_FR,CH_IT,AT_CH,DE_CH,FR_CH,IT_CH
datetime,,,,,,,,
2022-01-01 00:00:00,1200.0,4000.0,1400.0,3158.0,1200.0,800.0,3200.0,1910.0
2022-01-01 01:00:00,1200.0,4000.0,1400.0,3213.0,1200.0,800.0,3200.0,1910.0
2022-01-01 02:00:00,1200.0,4000.0,1400.0,2824.0,1200.0,800.0,3200.0,1910.0
2022-01-01 03:00:00,1200.0,4000.0,1400.0,2678.0,1200.0,800.0,3200.0,1910.0
2022-01-01 04:00:00,1200.0,4000.0,1400.0,2629.0,1200.0,800.0,3200.0,1910.0
...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,1200.0,4000.0,1200.0,1507.0,1000.0,800.0,3700.0,1810.0
2022-12-31 20:00:00,1200.0,4000.0,1200.0,1507.0,1000.0,800.0,3700.0,1810.0
2022-12-31 21:00:00,1200.0,4000.0,1200.0,1507.0,1000.0,800.0,3700.0,1810.0


In [58]:
# Data preprocessing steps
activeslosses = activeslosses.interpolate(method='polynomial', order=2)
activeslosses_hour = activeslosses.resample('H').sum()
Forecast_renew = Forecast_renew.interpolate(method='polynomial', order=2)
Forecast_temp_hourly = Forecast_temp.resample('H').asfreq()
Forecast_temp_hourly_lin = Forecast_temp_hourly.interpolate(method='linear')
Forecast_temp_hourly_poly = Forecast_temp_hourly.interpolate(method='polynomial', order=1)
NTC = NTC.interpolate(method='polynomial', order=2)

In [68]:
# Merge the dataframes
merged_df = pd.merge(activeslosses, Forecast_renew, left_index=True, right_index=True, how='outer')
merged_df = pd.merge(merged_df, Forecast_temp, left_index=True, right_index=True, how='outer')
merged_df = pd.merge(merged_df, NTC, left_index=True, right_index=True, how='outer')

In [73]:
merged_df = merged_df.reset_index()

data = merged_df

In [74]:
data = data.rename(columns={'Unnamed: 0': 'datetime'})

data['datetime'] = pd.to_datetime(data['datetime'])


# Drop the first column
data = data.drop(data.columns[0], axis=1)

In [75]:
data['month'] = data['datetime'].dt.month # Which month of the corresponding year
data['day_of_month'] = data['datetime'].dt.day # Which day of the corresponding month
#data['day_of_year'] = data['datetime'].dt.dayofyear # Which day of the corresponding year
#data['week_of_year'] = data['datetime'].dt.weekofyear # Which week of the corresponding year
data['day_of_week'] = data['datetime'].dt.dayofweek # Which day of the corresponding week of the each month
data["quarter"] = data['datetime'].dt.quarter
data["is_wknd"] = data['datetime'].dt.weekday // 4 # df.date.dt.weekday => Starts from '0' means '0' = 'Monday'. So, '// 4' will give '1' when day number equals
# to '5'(which corresponds 'Saturday') and '6'(which corresponds 'Sunday') and '0' for rest of them. Consequently this column will represent whether 
# the day is weekend or not
data['is_month_start'] = data['datetime'].dt.is_month_start.astype(int) # Is it starting of the corresponding month
data['is_month_end'] = data['datetime'].dt.is_month_end.astype(int)
data['hour'] = data['datetime'].dt.hour
data['var'] = data['MWh'].rolling(window=24*5).var()

season_list = []
hemisphere = 'Northern'
for month in data['month']:
    season = find_season(month, hemisphere)
    season_list.append(season)
    
data['season'] = season_list


In [76]:
countries = {
    'CH': 'Switzerland',
    'IT': 'Italy',
    'FR': 'France',
    'GER': 'Germany',
}

# For each country, add a new column with holiday names or NaN if not a holiday
for code, country in countries.items():
    holiday_dict = holidays.CountryHoliday(country, years=data['datetime'].dt.year.unique().tolist())
    data[country] = data['datetime'].apply(lambda x: 1 if x in holiday_dict else 0)


In [77]:
# Perform one-hot encoding on 'month' and 'weekday'

df_encoded = pd.get_dummies(data, columns=['Switzerland','Italy', 'France','Germany' ,'month', 'day_of_month', "day_of_week", "quarter", "is_wknd", 'is_month_start','is_month_end', 'season' ], prefix=['Switzerland','Italy', 'France','Germany' ,'month', 'day_of_month', "day_of_week", "quarter", "is_wknd", 'is_month_start','is_month_end', 'season'])

# remove datetime
df_encoded = df_encoded.drop('datetime', axis=1)


#for col in ['month', 'day_of_month', "day_of_week", "quarter", "is_wknd", 'is_month_start','is_month_end', 'season' ]:
#    df_encoded[col] = df_encoded[col].astype(bool)


In [78]:
input_seq

,solar_fore_de_mw,solar_fore_it_mw,wind_fore_de_mw,wind_fore_it_mw,temperature_fore_ch,temperature_fore_fr,temperature_fore_de,temperature_fore_it,CH_AT,CH_DE,...,is_wknd_0,is_wknd_1,is_month_start_0,is_month_start_1,is_month_end_0,is_month_end_1,season_Autumn,season_Spring,season_Summer,season_Winter
0,0.0,0.0,31805.65,1331.48,6.65,7.90,10.25,7.38,1200.0,4000.0,...,False,True,False,True,True,False,False,False,False,True
1,0.0,0.0,29880.67,1438.15,6.77,8.31,10.64,7.25,1200.0,4000.0,...,False,True,False,True,True,False,False,False,False,True
2,0.0,0.0,28826.75,1623.80,6.42,8.01,10.46,7.09,1200.0,4000.0,...,False,True,False,True,True,False,False,False,False,True
3,0.0,0.0,27631.75,1894.75,6.08,7.77,10.21,6.94,1200.0,4000.0,...,False,True,False,True,True,False,False,False,False,True
4,0.0,0.0,27128.00,2335.05,5.68,7.62,10.07,6.85,1200.0,4000.0,...,False,True,False,True,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.0,0.0,32304.12,3411.88,2.22,7.87,0.78,4.88,1200.0,4000.0,...,False,True,True,False,True,False,False,False,False,True
188,0.0,0.0,33191.38,2991.78,2.07,7.78,0.79,4.27,1200.0,4000.0,...,False,True,True,False,True,False,False,False,False,True
189,0.0,0.0,33930.55,2702.10,1.92,7.51,0.91,3.79,1200.0,4000.0,...,False,True,True,False,True,False,False,False,False,True
190,0.0,0.0,34514.90,2708.43,2.05,6.99,0.99,3.56,1200.0,4000.0,...,False,True,True,False,True,False,False,False,False,True


In [79]:
import numpy as np

for col in df_encoded.columns:
    if df_encoded[col].dtype == 'object':
        # Assuming the column's values can be converted to boolean
        df_encoded[col] = df_encoded[col].astype(bool)

data = df_encoded

# Define sequence lengths
input_seq_length = 7 * 24  # Previous 7 days
output_seq_length = 24     # Next 24 hours

# Initialize lists to store sequences
input_sequences = []
output_sequences = []

# Iterate over the data to create sequences
for i in range(0, len(data) - input_seq_length - output_seq_length + 1, output_seq_length):
    input_seq = data.iloc[i : i + input_seq_length + output_seq_length]
    input_seq.iloc[input_seq_length + output_seq_length -1-24 :input_seq_length + output_seq_length - 1]['MWh'] = np.nan
    output_seq = data.iloc[i + input_seq_length : i + input_seq_length + output_seq_length]['MWh']
    
    # Append the sequences to the lists
    input_sequences.append(input_seq)
    output_sequences.append(output_seq)

# Convert lists to NumPy arrays
input_sequences = np.array(input_sequences)
output_sequences = np.array(output_sequences)

/tmp/ipykernel_173472/2733691500.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_seq.iloc[input_seq_length + output_seq_length -1-24 :input_seq_length + output_seq_length - 1]['MWh'] = np.nan


In [80]:
input_val = input_sequences.reshape(input_sequences.shape[0], input_sequences.shape[1] * input_sequences.shape[2])

In [82]:
input_val

array([[146.054792, 0.0, 0.0, ..., False, False, True],
       [150.981065, 0.0, 0.0, ..., False, False, True],
       [215.603282, 0.0, 0.0, ..., False, False, True],
       ...,
       [122.219937, 0.0, 0.0, ..., False, False, True],
       [133.77160644000003, 0.0, 0.0, ..., False, False, True],
       [145.03873, 0.0, 0.0, ..., False, False, True]], dtype=object)

In [97]:
def mean_absolute_errors(models, input_test, output_test):
    errors = []
    for i in range(24):
        predictions = models[f'model_{i}'].predict(input_test)
        error = np.mean(abs(predictions - output_test[:, i]))
        print(error)
        errors.append(error)
    return np.mean(errors)

In [98]:

overall_mean = mean_absolute_errors(models, input_val, output_sequences)
print(f"Overall mean of the MAEs: {overall_mean}")

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
23.330939093664878
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
26.48302431581375
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
26.95565026227548
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
27.554132477664037
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
29.7242988555987
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
30.814328447538703
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
30.290201356939654
[Li

In [88]:
input_test.shape

(55, 17280)

In [95]:
input_val.shape

(358, 17280)

In [94]:
output_sequences.shape

(358, 24)

In [ ]:
def iterative_forecasting(model, input_data, steps=24):
    future_forecast = []
    current_input = list(input_data)

    for i in range(steps):
        forecasted_val = model.predict([current_input])[0]
        future_forecast.append(forecasted_val)

        # Prepare new input for the next prediction
        current_input.pop(0)  # Remove the oldest lag
        current_input.append(forecasted_val)  # Append the newly predicted value

    return future_forecast

# Use the last known data (from the test set) to start the iterative forecasting
starting_data = X_test.iloc[0].values
forecast = iterative_forecasting(model, starting_data, 24)

print(forecast)

In [ ]:
#optuna

def objective(trial):
    params = {
        "objective": "regression",
        "metric": "mae",
        "n_estimators": 1000,
        "verbosity": -1,
        "bagging_freq": 1,
        'device': 'gpu',
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        
    }

    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, Y_train)
    predictions = model.predict(X_val)
    rmse = mean_absolute_error(Y_val, predictions)
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

In [ ]:
print('Best hyperparameters:', study.best_params)
print('Best RMSE:', study.best_value)

Best hyperparameters: {'learning_rate': 0.015247440377194395, 'num_leaves': 13, 'subsample': 0.13740858380047208, 'colsample_bytree': 0.4167953910212117, 'min_data_in_leaf': 15}
Best RMSE: 0.5582819486587627

In [ ]:
def plot_lgb_importances(model, plot=False, num=10):
    gain = model.feature_importances_   # changed this line
    feat_imp = pd.DataFrame({
        'feature': model.booster_.feature_name(),  # changed this line
        'gain': 100 * gain / gain.sum()
    }).sort_values('gain', ascending=False)
    
    if plot:
        plt.figure(figsize=(10, 10))
        sns.barplot(x='gain', y='feature', data=feat_imp.head(num))
        plt.title('Feature Importances')
        plt.tight_layout()
        plt.show()
    else:
        print(feat_imp.head(num))

plot_lgb_importances(model, num=20, plot=True)


In [ ]:
rmse = mae(Y_val, predictions)
print(rmse)

In [ ]:
np.mean(abs(Y_val-predictions))

In [ ]:
predictions = model.predict(X_val)
predictions

In [ ]:
predictions = model.predict(X_train)
np.mean(abs(Y_train-predictions))